In [1]:
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.schema import Document
import json
import os
from dotenv import load_dotenv
load_dotenv()

False

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [3]:
def read_examples(filename: str):
    """
    Reads a JSON file containing code description examples
    and returns them as a list of dictionaries.
    """
    with open(filename, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data

In [4]:
examples = read_examples("code_descriptions.json")

In [12]:
hf_token = '' # --- add your token here 

In [6]:
from langchain_huggingface import HuggingFaceEndpointEmbeddings
doc_embedder = HuggingFaceEndpointEmbeddings(
    model="Qwen/Qwen3-Embedding-8B",
    task="feature-extraction",
    model_kwargs={"normalize": True},
    huggingfacehub_api_token = hf_token
    
)

In [8]:
def create_db_examples(examples):
    docs = []
    for doc in examples:
        # use the definition as the content, and keep the term (and letter) as metadata
        docs.append(
            Document(
                page_content=doc['page_content'],
                metadata=doc['metadata']
            )
        )
    vector_store_hf = Chroma.from_documents(
                                            docs,
                                            doc_embedder,
                                        )
    return vector_store_hf

In [9]:
examples = read_examples("code_descriptions.json")
vector_store_hf = create_db_examples(examples)

In [10]:
def get_example_of_visualizations(chroma,query: str) -> str:
    """
    Retrieves example visualizations related to the query.

    Parameters:
    - chroma: vector database with examples
    - query (str): The user's query about plotting.

    Returns:
    - str: The content of the most relevant example file.
    """

    # Perform a similarity search
    results = chroma.similarity_search_with_score(query, k=1)

    # Extract the most relevant document
    doc, score = results[0]

    # Construct the full path to the txt file
    file_name = doc.metadata['source'].lstrip('./')
    full_path = os.path.join('./', file_name)

    # Read and return the content of the txt file
    try:
        with open(full_path, 'r', encoding='utf-8') as file:
            content = file.read()
            return content
    except Exception as e:
        print(f"An error occurred while reading the file: {str(e)}")
        return ""  # Return empty string if error occurs


In [11]:
get_example_of_visualizations(vector_store_hf, 'one year rolling mean for salinity')

'import numpy as np\nimport pandas as pd\nimport matplotlib.pyplot as plt\nimport xarray as xr\n\nFILE_IN = "text.nc"\n\nds = xr.open_dataset(FILE_IN)\ntime = pd.to_datetime(ds["time"].values)\ny = pd.Series(ds["analysed_sst"].values, index=time).astype(float)\ny = y.interpolate(limit_direction="both")\n\nrun7  = y.rolling(window=7,  center=True, min_periods=1).mean()\nrun30 = y.rolling(window=30, center=True, min_periods=1).mean()\n\n\n# 1) Original & running means\nplt.figure(figsize=(12,4.5))\nplt.plot(y.index, y.values, label="Original")\nplt.plot(run7.index, run7.values, label="7-day running mean")\nplt.plot(run30.index, run30.values, label="30-day running mean")\nplt.title("SST: Original & Running Means")\nplt.xlabel("Date")\nplt.ylabel("Temperature")\nplt.legend()\nplt.tight_layout()\nplt.show()'